In [ ]:
# start coding here

from cellwhisperer.validation import SingleCellDataSetForValidationScoring
from cellwhisperer.config import config
import json

from pathlib import Path
dataset_processor = SingleCellDataSetForValidationScoring(celltypes=snakemake.params.celltypes,
                                                          dataset=Path(snakemake.input.dataset))
adata = dataset_processor.adata

In [ ]:
snakemake.params.celltypes

In [ ]:
# generating the 'true' conversations
def row_to_conversation(row):
    return {'id': row.name,
         'image': row.name,
         'conversations': [
             {'from': 'human',
           'value': f'{snakemake.params.question}\n<image>'},
          {'from': 'gpt',
           'value': snakemake.params.response_prefix + row[dataset_processor.celltype_obs_colname].replace(" b ", " B ").replace("nk ", "NK ").replace(" ii ", " II ").replace(" t ", " T ").replace("cd", "CD")}
                          ]}

conversations = adata.obs.sample(frac=1, random_state=42).groupby(dataset_processor.celltype_obs_colname).head(snakemake.params.num_cells_per_celltype).apply(row_to_conversation, axis=1).values.tolist()
conversations[0]

In [ ]:
with open(snakemake.output.evaluation_dataset, "w") as f:
    json.dump(conversations, f)